compare calibration result and simulation result in the instant loop detector

In [51]:
import os
print("Current directory:", os.getcwd())
HORNSGATAN_HOME = os.environ["HORNSGATAN_HOME"]
os.chdir(HORNSGATAN_HOME)
print("Current directory:", os.getcwd())

#number = len(data)
import pandas as pd
date = '2020-01-01'
detector = 'w2e_in'
number = 0
path  = "data/sim_data/"
if number<1:
    postfix = f"{detector}_{date}"
else:
    postfix = f"{detector}_{date}_{number}"

instantInductionLoop_filename_xml = f"{path}instantInductionLoop_{postfix}.xml"


Current directory: /home/kaveh/projects/Hornsgatan
Current directory: /home/kaveh/projects/Hornsgatan


In [52]:
import xml.etree.ElementTree as ET

# Parse the XML content of output_detectors.xml
tree_output = ET.parse(instantInductionLoop_filename_xml)
root_output = tree_output.getroot()

# Extract data from the XML
instant_out_data = []
for instant_out in root_output.findall('instantOut'):
    instant_out_data.append({
        'id': instant_out.get('id'),
        'time': float(instant_out.get('time')),
        'state': instant_out.get('state'),
        'vehID': instant_out.get('vehID'),
        'speed': float(instant_out.get('speed')),  # Convert speed to km/h
        'length': instant_out.get('length'),
        'type': instant_out.get('type'),
        'gap': instant_out.get('gap')
    })

# Convert to a DataFrame
df_instant_out = pd.DataFrame(instant_out_data)
df_instant_out = df_instant_out[df_instant_out["state"]=='enter']
# Save to CSV
#output_csv_file_instant_out = 'output_instant_out.csv'
#df_instant_out.to_csv(output_csv_file_instant_out, index=False)
#print(f"Data from 'output_detectors.xml' successfully converted to CSV and saved as '{output_csv_file_instant_out}'.")


In [53]:
df_instant_out

,id,time,state,vehID,speed,length,type,gap
0,w2e_in,1.577837e+09,enter,0_w2e_in,10.26,5.00,DEFAULT_VEHTYPE,None
2,w2e_in,1.577837e+09,enter,1_w2e_in,9.82,5.00,DEFAULT_VEHTYPE,2.52
4,w2e_in,1.577837e+09,enter,2_w2e_in,9.82,5.00,DEFAULT_VEHTYPE,1.25
7,w2e_in,1.577837e+09,enter,3_w2e_in,11.09,5.00,DEFAULT_VEHTYPE,9.71
9,w2e_in,1.577837e+09,enter,5_w2e_in,7.93,5.00,DEFAULT_VEHTYPE,20.72
...,...,...,...,...,...,...,...,...
24408,w2e_in,1.577923e+09,enter,10157_w2e_in,10.10,5.00,DEFAULT_VEHTYPE,1.25
24411,w2e_in,1.577923e+09,enter,10158_w2e_in,10.11,5.00,DEFAULT_VEHTYPE,1.25
24413,w2e_in,1.577923e+09,enter,10160_w2e_in,10.74,5.00,DEFAULT_VEHTYPE,4.16
24415,w2e_in,1.577923e+09,enter,10159_w2e_in,10.74,5.00,DEFAULT_VEHTYPE,1.23


In [54]:
data = pd.read_csv(f'data/calibration_data/calibrated_data_{postfix}.csv')
data.rename(columns={"veh_id": "vehID"}, inplace=True)
data.head()

,vehID,time_detector_sim,speed_detector_sim,speed_factor,time_detector_real,depart,departSpeed,speed_detector_real,delta_time,delta_speed
0,0_w2e_in,1.577837e+09,10.262976,1.232050,1577836811,1577836769,max,9.166667,0.10,1.096310
1,1_w2e_in,1.577837e+09,9.816909,1.178501,1577836814,1577836770,max,7.500000,0.21,2.316909
2,2_w2e_in,1.577837e+09,10.049487,1.206421,1577836815,1577836770,max,8.888889,0.25,1.160598
3,3_w2e_in,1.577837e+09,11.093025,1.331696,1577836826,1577836787,max,11.111111,0.09,-0.018086
4,4_w2e_in,1.577837e+09,8.030520,0.964048,1577836847,1577836793,max,7.777778,-0.16,0.252742


In [55]:
print(len(data), len(df_instant_out))

10162 10162


compare time error and speed error in loop detector between simulation result and calibration result

In [63]:
compare_df = pd.merge(df_instant_out[["vehID","time","speed"]],
            data[["vehID","time_detector_sim","speed_detector_sim"
                  ,"time_detector_real","speed_detector_real"]],on="vehID")
compare_df.head()




,vehID,time,speed,time_detector_sim,speed_detector_sim,time_detector_real,speed_detector_real
0,0_w2e_in,1.577837e+09,10.26,1.577837e+09,10.262976,1577836811,9.166667
1,1_w2e_in,1.577837e+09,9.82,1.577837e+09,9.816909,1577836814,7.500000
2,2_w2e_in,1.577837e+09,9.82,1.577837e+09,10.049487,1577836815,8.888889
3,3_w2e_in,1.577837e+09,11.09,1.577837e+09,11.093025,1577836826,11.111111
4,5_w2e_in,1.577837e+09,7.93,1.577837e+09,7.932751,1577836849,6.666667


In [ ]:
#compare_df = compare_df.head(19)

In [64]:
time_calib_error = compare_df["time_calib_error"]=(compare_df["time_detector_sim"]-compare_df["time_detector_real"])
abs(time_calib_error).mean()

np.float64(0.5222111503056386)

In [65]:
time_sim_error = compare_df["time_sim_error"] = compare_df["time"]-compare_df["time_detector_real"]
abs(time_sim_error).mean()

np.float64(1.9395689687210846)

In [66]:
speed_calib_error= compare_df["speed_calib_error"] =compare_df["speed_detector_sim"]-compare_df["speed_detector_real"]
abs(speed_calib_error).mean()

np.float64(1.0430913536949387)

In [67]:
speed_sim_error=compare_df["speed_sim_error"] =compare_df["speed"]-compare_df["speed_detector_real"]
abs(speed_sim_error).mean()

np.float64(1.2202188982921123)

In [68]:
import plotly.graph_objects as go

fig = go.Figure()

xbins=dict(
        start=-4,  # Bin start
        end=11,    # Bin end
        size=1    # Bin size (width of each bin)
    )
sim_MAE = round(abs(compare_df['time_sim_error']).mean(),2)
calib_MAE = round(abs(compare_df['time_calib_error']).mean(),2)

fig.add_trace(go.Histogram(x=compare_df['time_sim_error'], name=f"time_sim_error --> MAE = {sim_MAE}", opacity=0.75, xbins=xbins))
fig.add_trace(go.Histogram(x=compare_df['time_calib_error'], name=f"time_calib_error  --> MAE = {calib_MAE}", opacity=0.75, xbins=xbins))

fig.update_layout(
    barmode='group',  # Use 'overlay' if you want stacked look
    xaxis_title='time error (s)',
    yaxis_title='Count',
    title=f"error  =  simulate  -  real      ------------         {postfix}"
)

fig.show()
fig.write_html(f"diagram/hist_time_error_{postfix}_new.html")

In [69]:
print(compare_df['time_sim_error'].min(), compare_df['time_sim_error'].max())
print(compare_df['time_calib_error'].min(), compare_df['time_calib_error'].max())

-39.049999952316284 51.47000002861023
-10.019999980926514 5.0299999713897705


In [48]:
print(compare_df['speed_sim_error'].min(), compare_df['speed_sim_error'].max())
print(compare_df['speed_calib_error'].min(), compare_df['speed_calib_error'].max())

-0.275555555555556 2.4499999999999993
-0.2793832282569735 2.4456961448789105


In [49]:
import plotly.graph_objects as go

fig = go.Figure()

xbins=dict(
        start=-7,  # Bin start
        end=7,    # Bin end
        size=1    # Bin size (width of each bin)
    )
sim_MAE = round(abs(compare_df['speed_sim_error']).mean(),2)
calib_MAE = round(abs(compare_df['speed_calib_error']).mean(),2)
fig.add_trace(go.Histogram(x=compare_df['speed_sim_error'], name=f"speed_sim_error --> MAE = {sim_MAE}", opacity=0.75, xbins=xbins))
fig.add_trace(go.Histogram(x=compare_df['speed_calib_error'], name=f"speed_calib_error --> MAE = {calib_MAE}", opacity=0.75, xbins=xbins))

fig.update_layout(
    barmode='group',  # Use 'overlay' if you want stacked look
    xaxis_title='Value',
    yaxis_title='Count',
    title=f"error  =  simulate  -  real      ------------         {postfix}"
)

fig.show()
fig.write_html(f"diagram/hist_speed_error_{postfix}_new.html")

In [50]:
#import os
#os.environ["HORNSGATAN_HOME"]
